# Sample for Kubeflow TFJob SDK

This is a sample for Kubeflow TFJob SDK `kubeflow-tfjob`.

The notebook shows how to use Kubeflow TFJob SDK to create, get, wait, check and delete tfjob.

In [1]:
from kubernetes.client import V1PodTemplateSpec
from kubernetes.client import V1ObjectMeta
from kubernetes.client import V1PodSpec
from kubernetes.client import V1Container

from kubeflow.tfjob import constants
from kubeflow.tfjob import utils
from kubeflow.tfjob import V1ReplicaSpec
from kubeflow.tfjob import V1TFJob
from kubeflow.tfjob import V1TFJobSpec
from kubeflow.tfjob import TFJobClient

Define namespace where tfjob needs to be created to. If not specified, below function defines namespace to the current one where SDK is running in the cluster, otherwise it will deploy to default namespace.

In [2]:
namespace = utils.get_default_target_namespace()

### Define TFJob

The demo only creates a worker of TFJob to run mnist sample.

In [3]:
container = V1Container(
    name="tensorflow",
    image="gcr.io/kubeflow-ci/tf-mnist-with-summaries:1.0",
    command=[
        "python",
        "/var/tf_mnist/mnist_with_summaries.py",
        "--log_dir=/train/logs", "--learning_rate=0.01",
        "--batch_size=150"
        ]
)

worker = V1ReplicaSpec(
    replicas=2,
    restart_policy="Never",
    template=V1PodTemplateSpec(
        spec=V1PodSpec(
            containers=[container]
        )
    )
)

chief = V1ReplicaSpec(
    replicas=1,
    restart_policy="Never",
    template=V1PodTemplateSpec(
        spec=V1PodSpec(
            containers=[container]
        )
    )
)

ps = V1ReplicaSpec(
    replicas=1,
    restart_policy="Never",
    template=V1PodTemplateSpec(
        spec=V1PodSpec(
            containers=[container]
        )
    )
)

tfjob = V1TFJob(
    api_version="kubeflow.org/v1",
    kind="TFJob",
    metadata=V1ObjectMeta(name="mnist",namespace=namespace),
    spec=V1TFJobSpec(
        clean_pod_policy="None",
        tf_replica_specs={"Worker": worker,
                          "Chief": chief,
                          "PS": ps}
    )
)

### Create TFJob

In [4]:
tfjob_client = TFJobClient()
tfjob_client.create(tfjob, namespace=namespace)

{'apiVersion': 'kubeflow.org/v1',
 'kind': 'TFJob',
 'metadata': {'creationTimestamp': '2020-01-10T06:05:17Z',
  'generation': 1,
  'name': 'mnist',
  'namespace': 'default',
  'resourceVersion': '24815779',
  'selfLink': '/apis/kubeflow.org/v1/namespaces/default/tfjobs/mnist',
  'uid': '2d0ad671-336f-11ea-b6a8-00000a1001ee'},
 'spec': {'cleanPodPolicy': 'None',
  'tfReplicaSpecs': {'Chief': {'replicas': 1,
    'restartPolicy': 'Never',
    'template': {'spec': {'containers': [{'command': ['python',
         '/var/tf_mnist/mnist_with_summaries.py',
         '--log_dir=/train/logs',
         '--learning_rate=0.01',
         '--batch_size=150'],
        'image': 'gcr.io/kubeflow-ci/tf-mnist-with-summaries:1.0',
        'name': 'tensorflow'}]}}},
   'PS': {'replicas': 1,
    'restartPolicy': 'Never',
    'template': {'spec': {'containers': [{'command': ['python',
         '/var/tf_mnist/mnist_with_summaries.py',
         '--log_dir=/train/logs',
         '--learning_rate=0.01',
         '

### Get the created TFJob 

In [5]:
tfjob_client.get('mnist', namespace=namespace)

{'apiVersion': 'kubeflow.org/v1',
 'kind': 'TFJob',
 'metadata': {'creationTimestamp': '2020-01-10T06:05:17Z',
  'generation': 1,
  'name': 'mnist',
  'namespace': 'default',
  'resourceVersion': '24815814',
  'selfLink': '/apis/kubeflow.org/v1/namespaces/default/tfjobs/mnist',
  'uid': '2d0ad671-336f-11ea-b6a8-00000a1001ee'},
 'spec': {'cleanPodPolicy': 'None',
  'tfReplicaSpecs': {'Chief': {'replicas': 1,
    'restartPolicy': 'Never',
    'template': {'spec': {'containers': [{'command': ['python',
         '/var/tf_mnist/mnist_with_summaries.py',
         '--log_dir=/train/logs',
         '--learning_rate=0.01',
         '--batch_size=150'],
        'image': 'gcr.io/kubeflow-ci/tf-mnist-with-summaries:1.0',
        'name': 'tensorflow'}]}}},
   'PS': {'replicas': 1,
    'restartPolicy': 'Never',
    'template': {'spec': {'containers': [{'command': ['python',
         '/var/tf_mnist/mnist_with_summaries.py',
         '--log_dir=/train/logs',
         '--learning_rate=0.01',
         '

### Get the TFJob status, check if the TFJob has been started.

In [6]:
tfjob_client.get_job_status('mnist', namespace=namespace)

'Created'

### Wait for the specified job to finish

In [7]:
tfjob_client.wait_for_job('mnist', namespace=namespace, watch=True)

NAME                           STATE                TIME                          
mnist                          Created              2020-01-10T06:05:17Z          
mnist                          Running              2020-01-10T06:05:29Z          
mnist                          Running              2020-01-10T06:05:29Z          
mnist                          Running              2020-01-10T06:05:29Z          
mnist                          Running              2020-01-10T06:05:29Z          
mnist                          Running              2020-01-10T06:05:29Z          
mnist                          Running              2020-01-10T06:05:29Z          
mnist                          Succeeded            2020-01-10T06:07:49Z          


### Check if the TFJob succeeded

In [8]:
tfjob_client.is_job_succeeded('mnist', namespace=namespace)

True

### Get the TFJob training logs.

In [9]:
tfjob_client.get_logs('mnist', namespace=namespace)

The logs of Pod mnist-chief-0:
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Instructions for updating:
Please use tf.data to implement this functionality.
Instructions for updating:
Please use tf.data to implement this functionality.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
2020-01-10 06:05:42.333504: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/tensorflow/mnist/input_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/tensorflow/mnist/input_data/train-labels-idx1-ubyte.gz
Successfully 

### Delete the TFJob

In [10]:
tfjob_client.delete('mnist', namespace=namespace)

{'kind': 'Status',
 'apiVersion': 'v1',
 'metadata': {},
 'status': 'Success',
 'details': {'name': 'mnist',
  'group': 'kubeflow.org',
  'kind': 'tfjobs',
  'uid': '2d0ad671-336f-11ea-b6a8-00000a1001ee'}}